In [2]:
#Import necessary Libraries
import pandas as pd
import numpy as np

import string
import re
import os

import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import TextVectorization
from tensorflow import keras
from tensorflow.keras import layers

from params import *


In [3]:
my_df = pd.read_csv("Dataset/data.csv")
print("**********************")
my_df.head()



**********************


,english,spanish
0,Go.,Ve.
1,Go.,Vete.
2,Go.,Vaya.
3,Go.,Váyase.
4,Hi.,Hola.


In [4]:
my_df.info()
print("**********************")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118964 entries, 0 to 118963
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   english  118964 non-null  object
 1   spanish  118964 non-null  object
dtypes: object(2)
memory usage: 1.8+ MB
**********************


In [5]:
# This line of code modifies the "spanish" column of the DataFrame `my_df` by applying a lambda function to each item in the column..

my_df["spanish"] = my_df["spanish"].apply(lambda item: Get_Params.begin_token + " " + item + " " + Get_Params.end_token)

In [6]:
missing_values = my_df.isnull().sum()

print("Missing Values:")
print(missing_values)

my_df.head()

Missing Values:
english    0
spanish    0
dtype: int64


,english,spanish
0,Go.,[start] Ve. [end]
1,Go.,[start] Vete. [end]
2,Go.,[start] Vaya. [end]
3,Go.,[start] Váyase. [end]
4,Hi.,[start] Hola. [end]


In [7]:
# Define punctuation marks along with the Spanish inverted question mark ("¿") to be removed or replaced.
prepro_ = string.punctuation + "¿"

# Remove square brackets from the punctuation set.
prepro_ = prepro_.replace("[", "")
prepro_ = prepro_.replace("]", "")

# Print the updated set of punctuation marks.
print(prepro_)

# Define a function to perform Spanish text standardization by converting input strings to lowercase and removing specified punctuation marks.
def perform_spanish_standardization(input_string):
    lowercase = tf.strings.lower(input_string)  # Convert input string to lowercase.
    return tf.strings.regex_replace(lowercase, "[%s]" % re.escape(prepro_), "")  # Remove specified punctuation marks.

# Instantiate a TextVectorization layer for English text with specified parameters.
vect_eng = TextVectorization(
    max_tokens=Get_Params.vocab_size,  # Maximum vocabulary size.
    output_mode="int",  # Output integers corresponding to token indices.
    output_sequence_length=Get_Params.sequence_length,  # Output sequences of fixed length.
)

# Instantiate a TextVectorization layer for Spanish text with specified parameters.
vect_spa = TextVectorization(
    max_tokens=Get_Params.vocab_size,  # Maximum vocabulary size.
    output_mode="int",  # Output integers corresponding to token indices.
    output_sequence_length=Get_Params.sequence_length + 1,  # Output sequences of fixed length with an additional token.
    standardize=perform_spanish_standardization,  # Apply Spanish text standardization.
)

# Adapt the English TextVectorization layer to the English text data from the DataFrame.
vect_eng.adapt(list(my_df["english"]))

# Adapt the Spanish TextVectorization layer to the Spanish text data from the DataFrame after applying standardization.
vect_spa.adapt(list(my_df["spanish"]))

!"#$%&'()*+,-./:;<=>?@\^_`{|}~¿


